<a href="https://colab.research.google.com/github/LeograndeCode/Neural-Highlighting-of-Affordance-Regions/blob/Parte-2/notebook3D_PointCloud_v1.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive') #replace with drive.mount('/content/drive/', force_remount=True) if the drive has changed since last mount in order to force the remount
%cd /content/drive/MyDrive/Neural-Highlighting-of-Affordance-Regions/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Neural-Highlighting-of-Affordance-Regions


In [2]:
!pip install git+https://github.com/openai/CLIP.git
!pip install kaolin==0.17.0 -f https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-2.5.1_cu121.html

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-rw2k53pt
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-rw2k53pt
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
Looking in links: https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-2.5.1_cu121.html


In [3]:
#Avaiable clip modules
import clip
clip.available_models()


['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [29]:
import clip
import copy
import json
import kaolin as kal
import kaolin.ops.mesh
import numpy as np
import os
import sys
import random
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import torchvision.transforms.functional as F

# Pytorch3D install
import sys
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install iopath
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html


# Data structures and functions for rendering
from pytorch3d.structures import Meshes, Pointclouds
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVOrthographicCameras,
    PointsRasterizationSettings,
    PointsRenderer,
    PulsarPointsRenderer,
    PointsRasterizer,
    AlphaCompositor,
    NormWeightedCompositor
)
from pytorch3d.io import load_objs_as_meshes, save_ply

from itertools import permutations, product
from pathlib import Path
from tqdm import tqdm
from torch.autograd import grad
from torchvision import transforms
from utils import device

!pip install open3d
import open3d as o3d

width = 256
depth = 4
out_dim = 2
input_dim = 3
n_augs = 2      #default is 1

class NeuralHighlighter(nn.Module):
    def __init__(self):
        super(NeuralHighlighter, self).__init__()
        input_size = 3 #Dimension of the vertex
        output_size = 2 #Dimension of the output
                        #for the standard highlighter task there are only 2 classes: target region and not target region.
                        #we use the element of the output vector corresponding to the probability of belonging to the target
                        #region as the highlight probability described in the main paper.
        layers = []

        #See Appendix B (page 13)
        #first linear layer followed by ReLU and LayerNorm
        layers.append(nn.Linear(input_dim, width))
        layers.append(nn.ReLU())
        layers.append(nn.LayerNorm([width]))
        #other [depth] linear layers followed by ReLU and LayerNorm
        # -> changing the depth hyperparameter results in a deeper/shallower net
        # -> total depth (in terms of modules[Linear+ReLU+LayerNorm]) = [depth] + 1
        for i in range(depth):
            layers.append(nn.Linear(width, width))
            layers.append(nn.ReLU())
            layers.append(nn.LayerNorm([width]))
        #last linear layer followed by softmax in order to output probability-like values
        layers.append(nn.Linear(width, out_dim))
        layers.append(nn.Softmax(dim=1))

        self.mlp = nn.ModuleList(layers)
        self.model = self.mlp
        print(self.mlp)

    def forward(self, x):
        for layer in self.model:
            x = layer(x)
        return x

def get_clip_model(clipmodel):
    model, preprocess = clip.load(clipmodel, device=device)
    return model, preprocess

# ================== HELPER FUNCTIONS =============================

def save_final_results(log_dir, name, point_cloud, mlp, vertices, colors, renderer, background):
    mlp.eval()
    with torch.no_grad():
        probs = mlp(vertices)
        max_idx = torch.argmax(probs, 1, keepdim=True)

        # for renders
        one_hot = torch.zeros(probs.shape).to(device)
        one_hot = one_hot.scatter_(1, max_idx, 1)

        highlight = torch.tensor([204, 255, 0]).to(device)
        gray = torch.tensor([180, 180, 180]).to(device)
        colors = torch.stack((highlight / 255, gray / 255)).to(device)

        color_point_cloud(one_hot, point_cloud, colors)

        rendered_images = renderer(point_cloud).permute(0, 3, 1, 2)

        # Save the point cloud as a .ply file
        save_ply(os.path.join(log_dir, f"{name}_pointcloud.ply"), point_cloud.points_list()[0])

        # for mesh: Save the final colored mesh to .ply
        final_color = torch.zeros(vertices.shape[0], 3).to(device)
        final_color = torch.where(max_idx == 0, highlight, gray)

        # Convert final_color to numpy array and assign to mesh vertex colors
        mesh.vertex_colors = o3d.utility.Vector3dVector(final_color.cpu().numpy())

        # Use o3d.io.write_triangle_mesh to save the mesh
        o3d.io.write_triangle_mesh(os.path.join(log_dir, f"{name}_mesh.ply"), mesh)


        # Save rendered images (optional)
        save_renders(log_dir, 0, rendered_images, name='final_render.jpg')


def clip_loss(rendered_images, encoded_text, clip_transform, augment_transform, clip_model):
    if n_augs == 0:
        clip_image = clip_transform(rendered_images)
        encoded_renders = clip_model.encode_image(clip_image)
        encoded_renders = encoded_renders / encoded_renders.norm(dim=1, keepdim=True)
        if encoded_text.shape[0] > 1:
            loss = torch.cosine_similarity(torch.mean(encoded_renders, dim=0),
                                                torch.mean(encoded_text, dim=0), dim=0)
        else:
            loss = torch.cosine_similarity(torch.mean(encoded_renders, dim=0, keepdim=True),
                                                encoded_text)

    elif n_augs > 0:
        loss = 1.0 #original 0.0
        for _ in range(n_augs):
            augmented_image = augment_transform(rendered_images)
            encoded_renders = clip_model.encode_image(augmented_image)
            if encoded_text.shape[0] > 1:
                loss -= torch.cosine_similarity(torch.mean(encoded_renders, dim=0),
                                                    torch.mean(encoded_text, dim=0), dim=0)
            else:
                loss -= torch.cosine_similarity(torch.mean(encoded_renders, dim=0, keepdim=True),
                                                    encoded_text)
    return loss


"""
def clip_loss(prompt, rendered_images, clip_model ):
    # compute cosine similarity
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, preprocess = get_clip_model(clip_model)
    text_input = clip.tokenize(prompt).to(device)
    text_embedding = model.encode_text(text_input)
    # Make sure the text_embedding is 1D (flatten) for the dot product of the clip loss
    text_embedding = text_embedding.flatten()
    images_embeddings = []
    # Compute aggregate image rappresentation
    for image in rendered_images:
        with torch.no_grad():
            image = transforms.ToPILImage()(image)
            image_input = preprocess(image).unsqueeze(0).to(device)
            images_embeddings.append(model.encode_image(image_input))
    images_aggregate = torch.stack(images_embeddings).mean(dim=0)
    # Flatten the aggregate image embeddings
    images_aggregate = images_aggregate.flatten()

    cosine_similarity = torch.dot(images_aggregate, text_embedding) / (torch.norm(images_aggregate) * torch.norm(text_embedding))
    return 1 - cosine_similarity

"""

"""
def clip_loss_custom(encoded_text, rendered_images, model, preprocess):
    # Make sure the text_embedding is 1D (flatten) for the dot product of the clip loss
    text_embedding = encoded_text.flatten()
    images_embeddings = []
    # Compute aggregate image rappresentation
    for image in rendered_images:
        image = transforms.ToPILImage()(image)
        image_input = preprocess(image).unsqueeze(0).to(device)
        images_embeddings.append(model.encode_image(image_input))
    images_aggregate = torch.stack(images_embeddings).mean(dim=0)
    # Flatten the aggregate image embeddings
    images_aggregate = images_aggregate.flatten()

    cosine_similarity = torch.dot(images_aggregate, text_embedding) / (torch.norm(images_aggregate) * torch.norm(text_embedding))
    return 1 - cosine_similarity

"""

def save_renders(dir, i, rendered_images, name=None):
    if name is not None:
        torchvision.utils.save_image(rendered_images, os.path.join(dir, name))
    else:
        torchvision.utils.save_image(rendered_images, os.path.join(dir, 'renders/iter_{}.jpg'.format(i)))


Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py310_cu121_pyt251/download.html


In [30]:
# Rendering for point cloud settings
# Initialize a camera.
R, T = look_at_view_transform(20, 10, 0)
cameras = FoVOrthographicCameras(device=device, R=R, T=T, znear=0.01)

# Define the settings for rasterization and shading. Here we set the output image to be of size
# 512x512. As we are rendering images for visualization purposes only we will set faces_per_pixel=1
# and blur_radius=0.0. Refer to raster_points.py for explanations of these parameters.
raster_settings = PointsRasterizationSettings(
    image_size=224,
    radius = 0.003,
    points_per_pixel = 10,
    # Add this line to ensure 3 channels in the output image
    bin_size=0,  # This is necessary for the output to be a 3-channel image
)


# Create a points renderer by compositing points using an alpha compositor (nearer points
# are weighted more heavily). See [1] for an explanation.
rasterizer = PointsRasterizer(cameras=cameras, raster_settings=raster_settings)
renderer = PointsRenderer(
    rasterizer=rasterizer,
    compositor=AlphaCompositor()
)

# Custom point cloud coloring based on prediction

def segment2rgb(pred_class, colors):
    pred_rgb = torch.zeros(pred_class.shape[0], 3).to(device)
    for class_idx, color in enumerate(colors):
        pred_rgb += torch.matmul(pred_class[:,class_idx].unsqueeze(1), color.unsqueeze(0))

    return pred_rgb

def color_point_cloud(pred_class, sampled_point_cloud, colors):
    pred_rgb = segment2rgb(pred_class, colors)
    sampled_point_cloud.features = pred_rgb




In [32]:
from torch.optim.lr_scheduler import MultiStepLR

# Constrain most sources of randomness
# (some torch backwards functions within CLIP are non-determinstic)
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

render_res = 224
learning_rate = 0.0001
n_iter = 2500
res = 224
obj_path = 'data/horse.obj'
#output_dir = './output/'
clip_model_name = 'ViT-L/14'

device = "cuda" if torch.cuda.is_available() else "cpu"

#Path(os.path.join(output_dir, 'renders')).mkdir(parents=True, exist_ok=True)

objbase, extension = os.path.splitext(os.path.basename(obj_path))



# If the file is a .obj file create mesh and sample to create a point cloud
mesh = o3d.io.read_triangle_mesh(obj_path)
# open3d point cloud
point_cloud = mesh.sample_points_uniformly(10000)
# Extract points and colors
points = torch.tensor(np.asarray(point_cloud.points), dtype=torch.float32).to(device)
# Create a default gray color for all points
colors = torch.tensor([[0.7, 0.7, 0.7]] * points.shape[0], dtype=torch.float32).to(device)

point_cloud = Pointclouds(points=[points], features=[colors])

# Initialize variables
background = torch.tensor((1., 1., 1.)).to(device)

#log_dir = output_dir

# CLIP and Augmentation Transforms
clip_normalizer = transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))

clip_transform = transforms.Compose([
        transforms.Resize((res, res)),
        clip_normalizer
])

augment_transform = transforms.Compose([
        transforms.RandomResizedCrop(res, scale=(1, 1)),
        transforms.RandomPerspective(fill=1, p=0.8, distortion_scale=0.5),
        clip_normalizer
])

# MLP Settings
mlp = NeuralHighlighter().to(device)
optim = torch.optim.Adam(mlp.parameters(), learning_rate)

#introducing learning rate decay
#with the prompt horse/saddle the loss plateaus
#scheduler = StepLR(optim, step_size=300, gamma=0.1)

#scheduler = MultiStepLR(optim, milestones=[300, 1800], gamma=0.1)  # Decay a epoch 300 e 1800


# list of possible colors
rgb_to_color = {(204/255, 1., 0.): "highlighter", (180/255, 180/255, 180/255): "gray"}
color_to_rgb = {"highlighter": [204/255, 1., 0.], "gray": [180/255, 180/255, 180/255]}
full_colors = [[204/255, 1., 0.], [180/255, 180/255, 180/255]]
colors = torch.tensor(full_colors).to(device)

name = 'horse_d_{}_augs_{}'.format(depth, n_augs)

# --- Prompt ---
# encode prompt with CLIP
clip_model, preprocess = get_clip_model(clip_model_name)
#prompts = ['A 3D render of a gray horse with highlighted hat',
#           'A 3D render of a gray horse with highlighted shoes',
#           'A 3D render of a gray horse with highlighted saddle']
prompts = ['A 3D render of a gray horse with highlighted horseback']



for i, prompt in enumerate(prompts):

  output_dir = './output_{}_{}/'.format(name, i)
  Path(os.path.join(output_dir, 'renders')).mkdir(parents=True, exist_ok=True)
  log_dir = output_dir

  #here we compute the text encoding only once
  #if we put it inside the loss, we repeat n_iter times the same computation
  with torch.no_grad():
    text_input = clip.tokenize([prompt]).to(device)
    encoded_text = clip_model.encode_text(text_input)
    encoded_text = encoded_text / encoded_text.norm(dim=1, keepdim=True)

  vertices = point_cloud.points_list()[0].to(device)
  n_views = 5

  losses = []


  # Optimization loop
  for i in tqdm(range(n_iter)):
    optim.zero_grad()

    # predict highlight probabilities
    pred_class = mlp(vertices)


    # color point cloud
    sampled_point_cloud = point_cloud
    color_point_cloud(pred_class, sampled_point_cloud, colors)
    point_cloud = sampled_point_cloud

    # the CLIP model dimension is (1, 3, 224, 224)
    # instead the rendering of provides (1, 224, 224, 3)

    rendered_images = renderer(point_cloud).permute(0, 3, 1, 2)

    # Calculate CLIP Loss
    loss = clip_loss(rendered_images, encoded_text, clip_transform, augment_transform, clip_model)

    #loss = clip_loss_custom(encoded_text, rendered_images, clip_model, preprocess)
    loss.backward(retain_graph=True)

    optim.step()

    #LR decay
    #scheduler.step()

    # update variables + record loss
    with torch.no_grad():
        losses.append(loss.item())

    # report results
    if i % 100 == 0:
        print("Last 100 CLIP score: {}".format(np.mean(losses[-100:])))
        save_renders(log_dir, i, rendered_images)
        with open(os.path.join(log_dir, "training_info.txt"), "a") as f:
            f.write(f"For iteration {i}... Prompt: {prompt}, Last 100 avg CLIP score: {np.mean(losses[-100:])}, CLIP score {losses[-1]}\n")


  # save results
  save_final_results(log_dir, name, point_cloud, mlp, vertices, colors, renderer, background)

  # Save prompts
  with open(os.path.join(output_dir, 'prompt.txt'), "w") as f:
    f.write(prompt)
    f.write("\n")
    f.write("initial learning rate:")
    f.write(str(learning_rate))
    f.write("\n")
    f.write("n_iter:")
    f.write(str(n_iter))
    f.write("\n")
    f.write("n_augs:")
    f.write(str(n_augs))
    f.write("\n")
    f.write("clip_model:")
    f.write(clip_model_name)
    f.write("\n")
    f.write("depth:")
    f.write(str(depth))

ModuleList(
  (0): Linear(in_features=3, out_features=256, bias=True)
  (1): ReLU()
  (2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (3): Linear(in_features=256, out_features=256, bias=True)
  (4): ReLU()
  (5): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (6): Linear(in_features=256, out_features=256, bias=True)
  (7): ReLU()
  (8): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (9): Linear(in_features=256, out_features=256, bias=True)
  (10): ReLU()
  (11): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (12): Linear(in_features=256, out_features=256, bias=True)
  (13): ReLU()
  (14): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (15): Linear(in_features=256, out_features=2, bias=True)
  (16): Softmax(dim=1)
)


  0%|          | 1/2500 [00:00<07:06,  5.86it/s]

torch.Size([1, 3, 224, 224])
Last 100 CLIP score: 0.45263671875
torch.Size([1, 3, 224, 224])


  0%|          | 3/2500 [00:00<05:37,  7.39it/s]

torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])


...

torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])


100%|█████████▉| 2498/2500 [05:03<00:00,  8.43it/s]

torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])


100%|██████████| 2500/2500 [05:03<00:00,  8.24it/s]

torch.Size([1, 3, 224, 224])
